# Mercader Martínez, Francisco Javier

In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [2]:
semilla=80
np.random.seed(semilla)

# Entregable 2- Perceptrones y SVM.

## Machine Learning I. Grado en Ciencia de datos

Los siguientes ejercicios tienen que ser entregados dentro de las dos horas de prácticas. Como realizar el entregable:


-   La realización se debe de hacer de forma  **INDIVIDUAL**
-   Se debe de enviar un notebook con el código y las explicaciones, comentarios, análisis y justificaciones en Markdown.
-   Reproducibilidad:
    -   Se debe de establecer una variable semilla con un número al inicio, esta variables será la que se utilice para el np.random_seed y para el random_state.
    -   Las bases de datos se cargan con rutas relativas.
-   El notebook se debe de subir a la tarea del aula virtual creada antes de la finalización de la hora de clase. (Ver tarea y fecha de cierre).


⛔ Prohibido: no se puede utilizar ninguna herramienta de generación de código, no se permite el acceso a internet con ningún dispositivo.

### DATASET

El conjunto de datos Datos_crédito.csv describe los siguientes atributos:
- Edad: Edad de la persona en años.
- Sexo: Sexo de la persona: male o female.
- Trabajo: Tipo de trabajo. 0: sin formación y no residente, 1: sin formación y residente, 2: con formación, 3: formación alta.
- Vivienda: el régimen de posesión de la vivienda: own, free, rent.
- Ahorro: cantidad de dinero en la cuenta de ahorro: little, quite rich, rich, moderate.
- Corriente: cantidad de dinero en la cuenta corriente: 'little', 'moderate', 'rich'.
- Credito: cantidad de crédito permitido.
- Larga duracion: si el crédito es o no de larga duración. 0:no, 1:si.

## Ejercicio 1

Utilizando el dataset Datos_crédito.csv, cree el **perceptrón multicapa** que considere para realizar la **clasificación de la columna Larga duracion**. 


Para ello, siga los siguientes pasos:

1. Cargue el conjunto de datos (dataset) y revise los tipos de datos de los atributos del conjunto de datos, analiza si es necesario transformar alguno. En caso de ser necesario realiza la transformación.

In [3]:
data = pd.read_csv('Datos_crédito.csv')
print(data.dtypes) 

# Tenemos varias columnas que podemos convertir en variables categóricas

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

columns = ['Sexo', 'Vivienda', 'Ahorro', 'Corriente']

for column in columns:
    data[column] = encoder.fit_transform(data[column])
    
print(data.dtypes) 

Edad               int64
Sexo              object
Trabajo            int64
Vivienda          object
Ahorro            object
Corriente         object
Credito            int64
Larga duracion     int64
dtype: object
Edad              int64
Sexo              int32
Trabajo           int64
Vivienda          int32
Ahorro            int32
Corriente         int32
Credito           int64
Larga duracion    int64
dtype: object


2. Cree un hold-out (train-test) 75-25.

In [4]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['Larga duracion'], axis=1).values
y = data['Larga duracion']

X_ent, X_test, y_ent, y_test = train_test_split(X, y, test_size=0.25, random_state=semilla)

3. Aplique la estandarización de los datos para ajustar la escala de las características.

In [5]:
from sklearn.preprocessing import StandardScaler

# Instanciamos el escalador
scaler = StandardScaler()

# Estandarización del conjunto de entrenamiento
X_ent_escaladas = scaler.fit_transform(X_ent)

# Aplicamos la estandarización calculada con los datos de entrenamiento a los datos de test
X_test_escaladas = scaler.transform(X_test)

4. Cree un perceptrón multicapa dando valores a 4 hiperparámetros principales. Analice si existe un problema de subajuste o de sobreajuste o si, por el contrario no existe ningún problema.

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error

mlp = MLPClassifier(hidden_layer_sizes=(5), activation='logistic', 
                    solver='lbfgs', random_state=semilla)

mlp.fit(X_ent_escaladas, y_ent)

y_pred_ent = mlp.predict(X_ent_escaladas)
y_pred_test = mlp.predict(X_test_escaladas)

print(f"Error absoluto medio de entrenamiento: {mean_absolute_error(y_ent, y_pred_ent)}")
print(f"Error cuadratico medio de entrenamiento: {mean_squared_error(y_ent, y_pred_ent)}")
print(f"Error absoluto medio de test: {mean_absolute_error(y_test, y_pred_test)}")
print(f"Error cuadratico medio de test: {mean_squared_error(y_test, y_pred_test)}")

Error absoluto medio de entrenamiento: 0.20133333333333334
Error cuadratico medio de entrenamiento: 0.20133333333333334
Error absoluto medio de test: 0.264
Error cuadratico medio de test: 0.264


Los resultados del cálculo del MSE y el MAE tanto para el conjunto de entrenamiento como para el conjunto de test muestran valores *relativamente* bajos, lo que puede indicar que aunque `mlp` no es totalmente preciso tampoco comete demasiados errores.

5. Realice una operación de validación cruzada con el clasificador. ¿Corroboran los resultados obtenidos con la validación cruzada el resultado obtenido con el conjunto de test en el apartado 4? Razone la respuesta. 

In [7]:
from sklearn.model_selection import cross_val_score

puntuaciones = cross_val_score(mlp, X_ent_escaladas, y_ent, cv=5, scoring='accuracy')

print("Acierto de cada subconjunto: ", puntuaciones)
print(f"Media del acierto: {np.mean(puntuaciones):.4f}")
print(f"Desviación estándar del acierto: {np.std(puntuaciones):.4f}")

Acierto de cada subconjunto:  [0.68       0.69333333 0.65333333 0.67333333 0.72666667]
Media del acierto: 0.6853
Desviación estándar del acierto: 0.0244


La media de la precisión es relativamente alta (0.6853) y la desviación estándar es baja (0.0244), lo que indica que el modelo es bastante estable y robusto.

6. Realice una operación de búsqueda de los mejores hiperparámetros entre los utilizados en el apartado 4. Emplee una y dos capas, utilice dos números diferentes de neuronas en cada capa. 

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline = Pipeline([ ('transformador', scaler), ('MLP_CL', mlp)])

# Los valores de los hiperparámetros se guardan en un diccionario o en una lista de diccionarios
rejilla_hiperparametros = {
                            'MLP_CL__hidden_layer_sizes': [(50,), (20,), (50, 30), (45, 10)], 
                            'MLP_CL__activation': ['logistic', 'tanh', 'relu'], 
                            'MLP_CL__solver': ['lbfgs', 'adam']
                            }

# Número de folds para la validación cruzada
numero_subconjuntos = 5
# Función utilidad
funcion_utilidad = "accuracy"

# Se crean todas las configuraciones del perceptrón multicapa
grid_search = GridSearchCV(estimator=pipeline, 
                           param_grid=rejilla_hiperparametros, 
                           cv=numero_subconjuntos,
                           scoring=funcion_utilidad, 
                           refit=True,
                           return_train_score=True)

# Ejecutamos la búsqueda de hiperparámetros
grid_search.fit(X_ent_escaladas, y_ent)

# Mostramos los mejores parámetros
print(f"Mejores parámetros: {grid_search.best_params_}")

Mejores parámetros: {'MLP_CL__activation': 'logistic', 'MLP_CL__hidden_layer_sizes': (45, 10), 'MLP_CL__solver': 'adam'}


7. Obtenga la mejor configuración y sus métricas. Dado los valores de las métricas, ¿obtiene la configuración obtenida un rendimiento mejor que la configuración generada en el ejercicio 4? Razone la respuesta. Si el resultado ha cambiado, ¿qué influencia tienen los parámetros que han cambiado en el resultado?

In [9]:
print(f"Mejor configuración:  \n\t{grid_search.best_estimator_}")
print(f"\nMejor puntuación: {grid_search.best_score_}")

Mejor configuración:  
	Pipeline(steps=[('transformador', StandardScaler()),
                ('MLP_CL',
                 MLPClassifier(activation='logistic',
                               hidden_layer_sizes=(45, 10), random_state=80))])

Mejor puntuación: 0.7373333333333333


## Ejercicio 2

Utilizando el dataset Datos_crédito.csv, cree la **SVM no lineal** que considere para realizar la **regresión de la columna Edad**. 


Para ello, siga los siguientes pasos:

1. Cargue el conjunto de datos (dataset) y revise los tipos de datos de los atributos del conjunto de datos, analice si es necesario transformar alguno. En caso de ser necesario realiza la transformación.

In [10]:
data = pd.read_csv('Datos_crédito.csv')
print(data.dtypes) 

# Tenemos varias columnas que podemos convertir en variables categóricas

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

columns = ['Sexo', 'Vivienda', 'Ahorro', 'Corriente']

for column in columns:
    data[column] = encoder.fit_transform(data[column])

Edad               int64
Sexo              object
Trabajo            int64
Vivienda          object
Ahorro            object
Corriente         object
Credito            int64
Larga duracion     int64
dtype: object


2. Cree un hold-out (train-test) 70-30.

In [11]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['Edad'], axis=0)
y = data['Edad']

X_ent, X_test, y_ent, y_test = train_test_split(X, y, test_size=0.3, random_state=semilla)

3. Aplique la estandarización de los datos para ajustar la escala de las características.

In [12]:
from sklearn.preprocessing import StandardScaler

# Instanciamos el escalador
scaler = StandardScaler()

# Estandarización del conjunto de entrenamiento
X_ent_escaladas = scaler.fit_transform(X_ent)

# Aplicamos la estandarización calculada con los datos de entrenamiento a los datos de test
X_test_escaladas = scaler.fit_transform(X_test)

4. Cree una SVM no lineal y fije el valor de C igual a 1. Fije el valor de otros 2 hiperparámetros principales. Analice si existe un problema de subajuste o de sobreajuste o si, por el contrario no existe ningún problema.

In [13]:
# Importamos la clase Linear SVR
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Instancia de la SVM lineal
SVM_no_lineal_reg = SVR(kernel='rbf', gamma=1, epsilon=0.01, C=1)

# Entrenamos la SVM lineal de regresión
SVM_no_lineal_reg.fit(X_ent_escaladas, y_ent)

# Salidas de la SVM de regresión no lineal
y_pred_ent = SVM_no_lineal_reg.predict(X_ent_escaladas)
y_pred_test = SVM_no_lineal_reg.predict(X_test_escaladas)

print(f"Error absoluto medio de entrenamiento: {mean_absolute_error(y_ent, y_pred_ent)}")
print(f"Error cuadratico medio de entrenamiento: {mean_squared_error(y_ent, y_pred_ent)}")
print(f"Error absoluto medio de test: {mean_absolute_error(y_test, y_pred_test)}")
print(f"Error cuadratico medio de test: {mean_squared_error(y_test, y_pred_test)}")

Error absoluto medio de entrenamiento: 7.687537343573258
Error cuadratico medio de entrenamiento: 119.74402650017883
Error absoluto medio de test: 8.610527228431978
Error cuadratico medio de test: 125.19607086433516


Tanto los valores del MSE como del MAE en los conjuntos de entrenamiento y test son extremadamente altos, lo que implica que hay un problema de sobreajuste importante 

5. Ahora, manteniendo los hiperparámetros que ha fijado, obtenga los resultados para C=0,1 y C=100. ¿Cuál es el efecto de C sobre el rendimiento de la SVM? Razone la respuesta. 

In [14]:
for c in [0.1, 100]:
    print(f"C={c}")
    # Instancia de la SVM lineal
    SVM_no_lineal_reg = SVR(kernel='rbf', gamma=1, epsilon=0.01, C=c)

    # Entrenamos la SVM lineal de regresión
    SVM_no_lineal_reg.fit(X_ent_escaladas, y_ent)

    # Salidas de la SVM de regresión no lineal
    y_pred_ent = SVM_no_lineal_reg.predict(X_ent_escaladas)
    y_pred_test = SVM_no_lineal_reg.predict(X_test_escaladas)

    print(f"Error absoluto medio de entrenamiento: {mean_absolute_error(y_ent, y_pred_ent)}")
    print(f"Error cuadratico medio de entrenamiento: {mean_squared_error(y_ent, y_pred_ent)}")
    print(f"Error absoluto medio de test: {mean_absolute_error(y_test, y_pred_test)}")
    print(f"Error cuadratico medio de test: {mean_squared_error(y_test, y_pred_test)}")
    print()

C=0.1
Error absoluto medio de entrenamiento: 8.580314958276503
Error cuadratico medio de entrenamiento: 134.58147417227647
Error absoluto medio de test: 8.956554566720964
Error cuadratico medio de test: 131.63269181071757

C=100
Error absoluto medio de entrenamiento: 4.028190293550591
Error cuadratico medio de entrenamiento: 61.173408755716956
Error absoluto medio de test: 10.030368378903587
Error cuadratico medio de test: 165.64198165069806



Se puede observar que al aumentar el valor de $C$ el MSE y el MAE en el conjunto de entrenamiento disminuye, pero aumentan en el conjunto de test.

6. Realice una operación de búsqueda de los mejores hiperparámetros entre los utilizados en el apartado 4. 

In [15]:
# Definimos el Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVM_no_lineal_reg)
])

# Definimos la rejilla de hiperparámetros
param_grid = {
    'svr__gamma': [1, 5, 10, 20, 50, 100],
    'svr__C': [0.1, 1, 10, 100, 1000]
}

# Configuramos el GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', refit=True)

# Ejecutamos GridSearchCV
grid_search.fit(X_ent_escaladas, y_ent)

# Mostramos los mejores parámetros
print(f"Mejores parámetros: {grid_search.best_params_}")

Mejores parámetros: {'svr__C': 1, 'svr__gamma': 1}


7. Obtenga la mejor configuración y sus métricas de error. Dados los errores obtenidos, ¿obtiene la configuración obtenida un rendimiento mejor que la configuración generada en el ejercicio 4? Razone la respuesta. Si el resultado ha cambiado, ¿qué influencia tienen los parámetros que han cambiado en el resultado?

In [16]:
print(f"Mejor configuración: \n\t{grid_search.best_estimator_}")
print(f"\nMejor puntuación: {-grid_search.best_score_}")

Mejor configuración: 
	Pipeline(steps=[('scaler', StandardScaler()),
                ('svr', SVR(C=1, epsilon=0.01, gamma=1))])

Mejor puntuación: 133.524462316033
